# Label The Candidates! Extract The Features!

This notebook corresponds to labeling and genearting features for each extracted candidate from the [previous notebook](1.data-loader.ipynb).

## MUST RUN AT THE START OF EVERYTHING

Load all the imports and set up the database for database operations. Plus, set up the particular candidate type this notebook is going to work with. 

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from collections import defaultdict
import csv
import os
import re


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tqdm

In [2]:
#Set up the environment
username = "danich1"
password = "snorkel"
dbname = "pubmeddb"

#Path subject to change for different os
database_str = "postgresql+psycopg2://{}:{}@/{}?host=/var/run/postgresql".format(username, password, dbname)
os.environ['SNORKELDB'] = database_str

from snorkel import SnorkelSession
session = SnorkelSession()

In [3]:
from snorkel.annotations import FeatureAnnotator, LabelAnnotator
from snorkel.features import get_span_feats
from snorkel.models import candidate_subclass
from snorkel.models import Candidate, GoldLabel
from snorkel.viewer import SentenceNgramViewer

In [4]:
edge_type = "dg"
debug = False

In [5]:
if edge_type == "dg":
    DiseaseGene = candidate_subclass('DiseaseGene', ['Disease', 'Gene'])
    edge = "disease_gene"
elif edge_type == "gg":
    GeneGene = candidate_subclass('GeneGene', ['Gene1', 'Gene2'])
    edge = "gene_gene"
elif edge_type == "cg":
    CompoundGene = candidate_subclass('CompoundGene', ['Compound', 'Gene'])
    edge = "compound_gene"
elif edge_type == "cd":
    CompoundDisease = candidate_subclass('CompoundDisease', ['Compound', 'Disease'])
    edge = "compound_disease"
else:
    print("Please pick a valid edge type")

# Look at potential Candidates

Use this to look at loaded candidates from a given set. The constants represent the index to retrieve the appropiate set. Ideally, here is where one can look at a subset of the candidate and develop label functions for candidate labeling.

In [ ]:
#dev_set = pd.read_csv("vanilla_lstm/lstm_disease_gene_holdout/train_candidates_to_ids.csv")
#dev_set.head(3)

In [ ]:
#TRAIN = 0
#DEV = 1

In [ ]:
#candidates = session.query(DiseaseGene).filter(DiseaseGene.split==DEV).offset(300).limit(100)
#candidates = session.query(DiseaseGene).filter(DiseaseGene.id.in_(dev_set["id"])).offset(400).limit(100)
#sv = SentenceNgramViewer(candidates, session)

In [ ]:
#sv

# Label Functions

Here is one of the fundamental part of this project. Below are the label functions that are used to give a candidate a label of 1,0 or -1 which corresponds to correct label, unknown label and incorrection label. The goal here is to develop functions that can label accurately label as many candidates as possible. This idea comes from the [data programming paradigm](https://papers.nips.cc/paper/6523-data-programming-creating-large-training-sets-quickly), where the goal is to be able to create labels that machine learning algorithms can use for accurate classification.  

In [9]:
if edge_type == "dg":
    from utils.disease_gene_lf import LFS
elif edge_type == "gg":
    from utils.gene_gene_lf import *
elif edge_type == "cg":
    from utils.compound_gene_lf import *
elif edge_type == "cd":
    from utils.compound_disease_lf import *
else:
    print("Please pick a valid edge type")

/home/danich1/anaconda2/envs/snorkeling/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# Label The Candidates

Label each candidate based on the provided labels above. This code runs with realtive ease, but optimization is definitely needed when the number of label functions increases linearly.

In [10]:
from  sqlalchemy.sql.expression import func
labeler = LabelAnnotator(lfs=list(LFS.values()))

### Train Set

In [ ]:
sql = '''
SELECT id from candidate
WHERE split = 0 and type='disease_gene'
ORDER BY RANDOM()
LIMIT 50000;
'''
target_cids = [x[0] for x in session.execute(sql)]

In [ ]:
target_cids

In [ ]:
cids = session.query(DiseaseGene.id).filter(DiseaseGene.id.in_(target_cids))
%time L_train = labeler.apply(split=0, cids_query=cids, parallelism=5)

In [ ]:
np.savetxt('data/labeled_candidates.txt', target_cids)

### Dev Set

In [ ]:
sql = '''
SELECT candidate_id FROM gold_label
'''
gold_cids = [x[0] for x in session.execute(sql)]
gold_cids

In [11]:
sql = '''
SELECT id from candidate
WHERE split = 0 and type='disease_gene'
ORDER BY RANDOM()
LIMIT 10000;
'''
gold_cids = [x[0] for x in session.execute(sql)]
gold_cids

[11196390,
 6691154,
 3718558,
 32199854,
 10365368,
 26731372,
 34517639,
 31299484,
 28967497,
 6716878,
 31770736,
 22539824,
 30827155,
 19827256,
 36390261,
 2176714,
 34075897,
 24867987,
 32229006,
 20288074,
 27674474,
 5194638,
 34481280,
 30796279,
 20283686,
 19353970,
 31287759,
 28544792,
 29430883,
 25321861,
 31267939,
 35826272,
 32205988,
 35392803,
 29892314,
 15833209,
 30352907,
 25755526,
 1416538,
 30825579,
 22569655,
 33611834,
 23045128,
 20730698,
 14548973,
 36393022,
 15848313,
 29466814,
 16751671,
 25757579,
 29417504,
 19372251,
 17614435,
 27635071,
 31735057,
 2268481,
 29002941,
 4240356,
 2178759,
 24393749,
 21639315,
 18489895,
 22096868,
 1227925,
 2551318,
 19380308,
 32663841,
 26739201,
 34040283,
 34933749,
 2508582,
 11330039,
 27674759,
 7911940,
 28978390,
 34032626,
 31300733,
 16740194,
 33153301,
 15367137,
 2885715,
 15406889,
 912245,
 1079320,
 21211616,
 17161344,
 22549772,
 33574221,
 18069263,
 19839727,
 33111294,
 5184666,
 48592

In [12]:
cids = session.query(Candidate.id).filter(Candidate.id.in_(gold_cids))
%time L_dev = labeler.apply_existing(cids_query=cids, parallelism=5, clear=False)

Running UDF...
CPU times: user 1min 27s, sys: 2.36 s, total: 1min 29s
Wall time: 2min 26s


In [13]:
np.savetxt('data/labeled_dev_candidates.txt', gold_cids)

# DO NOT RUN BELOW

# Generate Candidate Features

In conjunction with each candidate label, generate candidate features that will be used by some machine learning algorithms (notebook 4). This step is broken as insert takes an **incredibly** long time to run. Had to do roundabout way to load the features. **Do not run this block** and refer to the code block below. Gonna need to debug this part, when I get time.

In [ ]:
%%time
featurizer = FeatureAnnotator()
featurizer.apply(split=0, clear=False)

F_dev = featurizer.apply_existing(split=1, parallelism=5, clear=False)
F_test = featurizer.apply_existing(split=2, parallelism=5, clear=False)

# Work Around for above code

As mentioned above this code is the workaround for the broken featurizer. The intuition behind this section is to write all the generated features to a sql text file. Exploting the psql's COPY command, the time taken for inserting features drops to ~30 minutues (compared to 1 week+).

In [ ]:
%%time

group = 0
chunksize = 1e5
seen = set()
feature_key_hash = defaultdict(int)
feat_counter = 0

with open('feature_key.sql', 'wb') as f:
    with open('feature.sql', 'wb') as g:
        # Write the headers
        f.write("COPY feature_key(\"group\", name, id) from stdin with CSV DELIMITER '	' QUOTE '\"';\n")
        g.write("COPY feature(value, candidate_id, key_id) from stdin with CSV DELIMITER '	' QUOTE '\"';\n")
        
        # Set up the writers
        feature_key_writer = csv.writer(f, delimiter='\t',  quoting=csv.QUOTE_NONNUMERIC)
        feature_writer = csv.writer(g, delimiter='\t', quoting=csv.QUOTE_NONNUMERIC)
        
        # For each split get and generate features
        for split in [0,1,2]:
    
            #reset pointer to cycle through database again
            pointer = 0
            
            print(split)
            candidate_query = session.query(Candidate).filter(Candidate.split==split).limit(chunksize)
            
            while True:
                candidates = candidate_query.offset(pointer).all()
                
                if not candidates:
                    break

                for c in tqdm.tqdm(candidates):
                    try:
                        for name, value in get_span_feats(c):

                            # If the training set, set the feature hash
                            if split == 0:
                                if name not in feature_key_hash:
                                    feature_key_hash[name] = feat_counter
                                    feat_counter = feat_counter + 1
                                    feature_key_writer.writerow([group, name, feature_key_hash[name]])

                            if name in feature_key_hash:
                                # prevent duplicates from being written to the file
                                if (c.id, name) not in seen:
                                    feature_writer.writerow([value, c.id, feature_key_hash[name]])
                                    seen.add((c.id, name))

                        #To prevent memory overload
                        seen = set()
                    
                    except Exception as e:
                        print(e.message)
                        print(c)
                        print(c.get_parent().text)

                # update pointer for database
                pointer = pointer + chunksize

# Generate Coverage Stats

Before throwing our labels at a machine learning algorithm take a look at some quick stats. The code below will show the coverage and conflicts of each label function. Furthermore, this code will show the dimension of each label matrix.

In [ ]:
print(L_train.lf_stats(session, ))

In [ ]:
print(L_dev.lf_stats(session, ))